In [33]:
import numpy as np
import sklearn as sk
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from sqlite_api import *

In [37]:
fdb = 'fantasy.db'

player_lbls = ['Player', 'Position', 'Team', 'Season', 'Plays', 'Games', 'RushAttempts', 
              'RushYrds', 'RushTDs', 'PassAttempts', 'Complete', 'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions']

In [38]:
## Scoring metrics

qb = {'PassYrds': 0.04, # QBs get 0.04 points per passing yard 
        'PassTDs': 4, 
        'Interceptions': -2,
        'RushYrds': 0.1,
        'RushTDs': 6,
        'Fumbles': -2}

wr = {'PassYrds': 0.01,
         'PassTDs': 6,
         'Interceptions': -2,
         'RushYrds': 0.1,
         'RushTDs': 6,
         'Fumbles': -2}

rb = wr.copy()

te = wr.copy()

scores = {'QB': qb,
             'RB': rb,
             'WR': wr,
             'TE': te}

# need 2 points for 2point conversions

In [39]:
def score_row(row, qb_scores, wr_scores, rb_scores, te_scores):
    score = 0
    if row['Position'] == 'QB':
        metrics = qb_scores
    elif row['Position'] == 'WR':
        metrics = wr_scores
    elif row['Position'] == 'RB':
        metrics = rb_scores
    elif row['Position'] == 'TE':
        metrics = te_scores
    
    for var in metrics:
        score += row[var] * metrics[var]
        
    return score

In [75]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls, where='WHERE Season=2016')  

In [86]:
# create 2016 mock season database
rdb = 'rankings.db'
conn = sqlite3.connect(rdb)
c = conn.cursor()

,Player,Position,Team
0,Aaron Rodgers,QB,GB
1,Drew Brees,QB,NO
2,Matt Ryan,QB,ATL
3,Andrew Luck,QB,IND
4,Kirk Cousins,QB,WAS


In [95]:
c.execute('DROP TABLE Kickers')

OperationalError: no such table: Kickers

In [96]:
for pos in ['Quarterbacks', 'RunningBacks', 'WideReceivers', 'TightEnds', 'Kickers']:
    c.execute('''CREATE Table ''' + pos +
                '''(Player VARCHAR(30),
                    Team VARCHAR(3),
                    Position CHARACTER(2),
                    PRIMARY KEY (Player))''')

In [97]:
c.execute('''CREATE Table Defenses
                (Team VARCHAR(3),
                Position CHARACTER(2),
                PRIMARY KEY (Team))''')

In [102]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls, where='WHERE Season=2016')[['Player', 'Position', 'Team']]
QBs = df[df.Position == 'QB']
for idx in range(len(QBs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO Quarterbacks VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [103]:
RBs = df[df.Position == 'RB']
for idx in range(len(RBs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO RunningBacks VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [104]:
WRs = df[df.Position == 'WR']
for idx in range(len(WRs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO WideReceivers VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [105]:
TEs = df[df.Position == 'TE']
for idx in range(len(TEs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO TightEnds VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [107]:
df = select_to_df(fdb, 'KickerSeason', ['Player', 'Team', 'Position'], 'WHERE Season = 2016')
for idx in range(len(df)):
    c.execute('INSERT INTO Kickers VALUES (?, ?, ?)', df.iloc[idx])

In [108]:
df = select_to_df(fdb, 'DefenseSeason', ['Team', 'Position'], 'WHERE Season = 2016')
for idx in range(len(df)):
    c.execute('INSERT INTO Defenses VALUES (?, ?)', df.iloc[idx])

In [109]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls, 'WHERE Season != 2016')
df['Score'] = df.apply(score_row, args=(qb, wr, rb, te), axis=1)

In [111]:
df.tail()

,Player,Position,Team,Season,Plays,Games,RushAttempts,RushYrds,RushTDs,PassAttempts,Complete,PassYrds,PassTDs,Fumbles,Interceptions,Score
5365,MarQueis Gray,TE,BUF,2015,1.0,4.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.02
5366,Matthew Mulligan,TE,BUF,2015,1.0,12.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.02
5367,Asante Cleveland,TE,NE,2015,1.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.01
5368,Alex Smith,TE,WAS,2015,2.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.00
5369,Geoff Swaim,TE,DAL,2015,1.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.00


## Adding Independent Features

In [112]:
def pass_tdatt_rat(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['PassTDs'] / row['PassAttempts']

def rush_tdatt_rat(row):
    if row['RushAttempts'] == 0:
        return 0
    return row['RushTDs'] / row['RushAttempts']

def complete_perc(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['Complete'] / row['PassAttempts']

def avg_rush_yrds(row):
    if row['RushAttempts'] == 0:
        return 0
    return row['RushYrds'] / row['RushAttempts']

def avg_pass_yrds(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['PassYrds'] / row['PassAttempts']

def avg_plays(row):
    return row['Plays'] / row['Games']

def score_per_play(row):
    return row['Score'] / row['Plays']

def score_per_game(row):
    return row['Score'] / row['Games']

In [113]:
df['PassTDAttRatio'] = df.apply(pass_tdatt_rat, axis=1)
df['RushTDAttRatio'] = df.apply(rush_tdatt_rat, axis=1)
df['AvgPassYrds'] = df.apply(avg_pass_yrds, axis=1)
df['AvgRushYrds'] = df.apply(avg_rush_yrds, axis=1)
df['PercentComplete'] = df.apply(complete_perc, axis=1)
df['PlaysPerGame'] = df.apply(avg_plays, axis=1)
df['ScorePerPlay'] = df.apply(score_per_play, axis=1)
df['ScorePerGame'] = df.apply(score_per_game, axis=1)

In [121]:
df[(df.PassAttempts > 20) & (df.Position == 'QB')].sort_values('AvgPassYrds', ascending=False)

,Player,Position,Team,Season,Plays,Games,RushAttempts,RushYrds,RushTDs,PassAttempts,...,Interceptions,Score,PassTDAttRatio,RushTDAttRatio,AvgPassYrds,AvgRushYrds,PercentComplete,PlaysPerGame,ScorePerPlay,ScorePerGame
2687,Matt Flynn,QB,GB,2011,62.0,5.0,13.0,-6.0,1.0,49.0,...,2.0,44.12,0.122449,0.076923,10.571429,-0.461538,0.673469,12.400000,0.711613,8.824000
3222,Kirk Cousins,QB,WAS,2012,51.0,3.0,3.0,22.0,0.0,48.0,...,3.0,30.84,0.083333,0.000000,9.708333,7.333333,0.687500,17.000000,0.604706,10.280000
1638,Tarvaris Jackson,QB,MIN,2009,38.0,8.0,17.0,-10.0,0.0,21.0,...,0.0,11.04,0.047619,0.000000,9.571429,-0.588235,0.666667,4.750000,0.290526,1.380000
4862,Landry Jones,QB,PIT,2015,60.0,7.0,5.0,-5.0,0.0,55.0,...,4.0,22.02,0.054545,0.000000,9.327273,-1.000000,0.581818,8.571429,0.367000,3.145714
45,Charlie Batch,QB,PIT,2006,66.0,8.0,13.0,15.0,0.0,53.0,...,0.0,39.18,0.094340,0.000000,9.283019,1.153846,0.584906,8.250000,0.593636,4.897500
2646,Aaron Rodgers,QB,GB,2011,562.0,15.0,60.0,257.0,3.0,502.0,...,6.0,397.42,0.089641,0.050000,9.249004,4.283333,0.683267,37.466667,0.707153,26.494667
3747,Nick Foles,QB,PHI,2013,374.0,13.0,57.0,221.0,3.0,317.0,...,2.0,255.74,0.085174,0.052632,9.119874,3.877193,0.640379,28.769231,0.683797,19.672308
1100,Matt Leinart,QB,ARI,2008,33.0,4.0,4.0,5.0,0.0,29.0,...,1.0,11.06,0.034483,0.000000,9.103448,1.250000,0.517241,8.250000,0.335152,2.765000
50,A.J. Feeley,QB,PHI,2006,39.0,2.0,1.0,3.0,0.0,38.0,...,0.0,23.98,0.078947,0.000000,9.000000,3.000000,0.684211,19.500000,0.614872,11.990000
44,Kyle Boller,QB,BAL,2006,77.0,5.0,22.0,34.0,0.0,55.0,...,2.0,36.80,0.090909,0.000000,8.818182,1.545455,0.600000,15.400000,0.477922,7.360000


In [124]:
select_to_df(rdb, 'Quarterbacks', ['Player', 'Team'])

,Player,Team
0,Aaron Rodgers,GB
1,Drew Brees,NO
2,Matt Ryan,ATL
3,Andrew Luck,IND
4,Kirk Cousins,WAS
5,Philip Rivers,LAC
6,Matthew Stafford,DET
7,Blake Bortles,JAC
8,Dak Prescott,DAL
9,Jameis Winston,TB


In [123]:
# separate models for each position
# how to factor in new team with offense stats
conn.commit()

In [ ]:
# elo match up system